1. Declaring our sample (date)

In [1]:
# Sample. The number of cases of 0,1-5,6,7-11 and 12 votes for aquittal.
# Each tuple includes the assumed average number of votes for acquittal(1) and the number of cases that had that number of votes(2)
# We assume the average number of votes for the ranges 1-5 and 7-11 is 3 and 9
X = [(0,43),(3,105),(6,10),(9,41),(12,26)]
N = 225

M_1 = sum(X[i][0]*X[i][1] for i in range(0,5))/N
m_2 = sum((X[i][0]-M_1)^2*X[i][1] for i in range(0,5))/N
M_3 = sum(X[i][0]^3*X[i][1] for i in range(0,5))/N

2. Method of moments

In [2]:
from scipy.optimize import root
from scipy.optimize import least_squares

n = 12
var("th", "mu", "thh", "mu1", "mu2")

f1 = th*n*(1-mu) + (1-th)*n*mu - M_1
f2 = n*( (mu*(1-mu)) + (th*(1-th)*n*(2*mu-1)) ) - m_2

g1 = thh*n*(1-mu1) + (1-thh)*n*mu2 - M_1
g2 = thh*n*(1-mu1)*mu1 + thh*n^2*(1-mu1)^2 + (1-thh)*n*mu2*(1-mu2) + (1-thh)*n^2*mu2^2 - thh^2*n^2*(1-mu1)^2 - (1-thh)^2*n^2*mu2^2 - 2*thh*(1-thh)*n^2*(1-mu1)*mu2 - m_2
g3 = thh*(n*(1-mu1) + 3*n*(n-1)*(1-mu1)^2*mu1^2 + n*(n-1)*(n-2)*(1-mu1)^3) + (1-thh)*(n*mu2 + 3*n*(n-1)*mu2^2*(1-mu2)^2 + n*(n-1)*(n-2)*mu2^3) - M_3

F = lambda vf: vector([f1.subs(th=vf[0], mu=vf[1]), f2.subs(th=vf[0], mu=vf[1])])
G = lambda vg: vector([g1.subs(thh=vg[0], mu1=vg[1], mu2=vg[2]), g2.subs(thh=vg[0], mu1=vg[1], mu2=vg[2]), g3.subs(thh=vg[0], mu1=vg[1], mu2=vg[2])])

vf0 = vector([0.75, 0.75])
vg0 = vector([0.75, 0.8, 0.1])

solf = root(F, vf0)
solg = root(G, vg0)

print(solf, solg)

    fjac: array([[-0.1734627 , -0.98484044],
       [ 0.98484044, -0.1734627 ]])
     fun: array([-2.50789611e-10, -1.86822113e-09])
 message: 'The solution converged.'
    nfev: 8
     qtf: array([-1.54729278e-07, -6.65242377e-09])
       r: array([ 31.46956607, -47.83849925, -14.30108497])
  status: 1
 success: True
       x: array([0.73722516, 0.72950535])     fjac: array([[-1.17357606e-08,  6.64728875e-03, -9.99977907e-01],
       [-6.35532459e-03,  9.99957712e-01,  6.64715458e-03],
       [-9.99979805e-01, -6.35518426e-03, -4.22339424e-05]])
     fun: array([  -2.08723089,  -13.01598355, -326.30773786])
 message: 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.'
    nfev: 25
     qtf: array([326.21400762, -15.17118607,   2.18368897])
       r: array([ 4.84854847e+10,  3.73591831e+11, -2.38533261e+10,  2.98808347e+04,
        1.21207574e+03, -1.37409793e-02])
  status: 5
 success: False
       x: array([0.9379417 , 0.79046

In [3]:
vf0 = vector([0.7, 0.7])
vg0 = vector([0.45, 0.7, 0.7])

solf = root(F, vf0)
solg = root(G, vg0)

print(solf, solg)

    fjac: array([[-0.17266789, -0.9849801 ],
       [ 0.9849801 , -0.17266789]])
     fun: array([-4.38875603e-10, -3.43172468e-09])
 message: 'The solution converged.'
    nfev: 9
     qtf: array([2.25793895e-07, 1.05114203e-08])
       r: array([ 32.06717379, -51.45869389, -14.42546996])
  status: 1
 success: True
       x: array([0.73722516, 0.72950535])     fjac: array([[-0.00364154, -0.03180878, -0.99948734],
       [ 0.25277102, -0.96706537,  0.029856  ],
       [ 0.96751928,  0.25253271, -0.01156194]])
     fun: array([-1.27895472e-11, -2.30128805e-10, -3.19147375e-10])
 message: 'The solution converged.'
    nfev: 88
     qtf: array([-1.15187685e-07, -5.60622505e-08,  1.78066481e-08])
       r: array([ 3.14736878e+03,  3.74685539e+02, -7.21460269e+02, -3.69549823e+01,
        2.74636030e+00, -2.49665185e+00])
  status: 1
 success: True
       x: array([0.90486645, 0.70792718, 1.33311769])


3. Maximum Likelihood Estimation method

In [4]:
from scipy.optimize import differential_evolution

def gamma2(n,i, th, mu):
    return binomial(n,i)*(th*(mu^(n-i)*(1-mu)^i + (1-th)*(1-mu)^(n-i)*mu^i))

def gamma3(n,i, thh, mu1, mu2):
    return binomial(n,i)*(thh*(mu1^(n-i)*(1-mu1)^i + (1-thh)*(1-mu2)^(n-i)*mu2^i))

# We omit the constant log(225!/(43!105!10!41!26!)) of the logarithm of the likelihood function because it does not change the minimum, yet it complicates the expression and is unnecessary.
def neg_log_likelihood2(params2):
    th, mu = params2
    log_lik = X[0][1]*log(gamma2(12,0,th,mu)) + X[1][1]*(sum(log(gamma2(12,i,th,mu)) for i in range(1,6))) + X[2][1]*log(gamma2(12,6,th,mu)) + X[3][1]*(sum(log(gamma2(12,i,th,mu)) for i in range(7,12))) + X[4][1]*log(gamma2(12,12,th,mu))
    return -log_lik  

def neg_log_likelihood3(params3):
    thh, mu1, mu2 = params3
    log_lik = X[0][1]*log(gamma3(12,0,thh,mu1,mu2)) + X[1][1]*(sum(log(gamma3(12,i,thh,mu1,mu2)) for i in range(1,6))) + X[2][1]*log(gamma3(12,6,thh,mu1,mu2)) + X[3][1]*(sum(log(gamma3(12,i,thh,mu1,mu2)) for i in range(7,12))) + X[4][1]*log(gamma3(12,12,thh,mu1,mu2))
    return -log_lik  


bounds2 = [(0.5, 1.0), (0.5, 1.0)]
bounds3 = [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

result2 = differential_evolution(
    neg_log_likelihood2,
    bounds2,
    tol=1e-10,
    strategy='best1bin',
    polish=True,
    updating='deferred',
    seed=42
)

result3 = differential_evolution(
    neg_log_likelihood3,
    bounds3,
    tol=1e-10,
    strategy='best1bin',
    polish=True,
    updating='deferred',
    seed=42
)

print("Success:", result2.success)
print("Params:", result2.x)

print("Success:", result3.success)
print("Params:", result3.x)

Success: True
Params: [0.78570868 0.76430702]
Success: True
Params: [0.79474869 0.7506057  0.79617205]


4. Minimum chi^2 method

In [5]:
from scipy.optimize import differential_evolution

def gamma2(n,i, th, mu):
    return binomial(n,i)*(th*(mu^(n-i)*(1-mu)^i + (1-th)*(1-mu)^(n-i)*mu^i))

def gamma3(n,i, thh, mu1, mu2):
    return binomial(n,i)*(thh*(mu1^(n-i)*(1-mu1)^i + (1-thh)*(1-mu2)^(n-i)*mu2^i))

def chi2_2(params2):
    th, mu = params2
    p0 = gamma2(12,0, th, mu)
    p1 = sum(gamma2(12,i, th, mu) for i in range(1,6))
    p2 = gamma2(12,6, th, mu)
    p3 = sum(gamma2(12,i, th, mu) for i in range(7,12))
    p4 = gamma2(12,12, th, mu)
    chi2 = (X[0][1]/N - p0)^2/p0 + (X[1][1]/N - p1)^2/p1 + (X[2][1]/N - p2)^2/p2 + (X[3][1]/N - p3)^2/p3 + (X[4][1]/N - p4)^2/p4
    return chi2

def chi2_3(params3):
    thh, mu1, mu2 = params3
    p03 = gamma3(12,0, thh, mu1, mu2)
    p13 = sum(gamma3(12,i, thh, mu1, mu2) for i in range(1,6))
    p23 = gamma3(12,6, thh, mu1, mu2)
    p33 = sum(gamma3(12,i, thh, mu1, mu2) for i in range(7,12))
    p43 = gamma3(12,12, thh, mu1, mu2)
    chi2 = (X[0][1]/N - p03)^2/p03 + (X[1][1]/N - p13)^2/p13 + (X[2][1]/N - p23)^2/p23 + (X[3][1]/N - p33)^2/p33 + (X[4][1]/N - p43)^2/p43
    return chi2  


bounds2 = [(0.5, 1.0), (0.5, 1.0)]
bounds3 = [(0.5, 1.0), (0.5, 1.0), (0.5, 1.0)]

result2 = differential_evolution(
    chi2_2,
    bounds2,
    tol=1e-12,
    strategy='best1bin',
    polish=True,
    updating='deferred',
    seed=42
)

result3 = differential_evolution(
    chi2_3,
    bounds3,
    tol=1e-12,
    strategy='best1bin',
    polish=True,
    updating='deferred',
    seed=42
)

print("Success:", result2.success)
print("Params:", result2.x)

print("Success:", result3.success)
print("Params:", result3.x)

Success: True
Params: [0.65639282 0.83971256]
Success: True
Params: [0.73096942 0.820267   0.92408853]


5. Likelihood of observing the sample with both models

In [6]:
th = 0.78570868
mu = 0.76430702
thh = 0.79474869
mu1 = 0.7506057
mu2 = 0.79617205

p0 = gamma2(12,0, th, mu)
p1 = sum(gamma2(12,i, th, mu) for i in range(1,6))
p2 = gamma2(12,6, th, mu)
p3 = sum(gamma2(12,i, th, mu) for i in range(7,12))
p4 = gamma2(12,12, th, mu)

p03 = gamma3(12,0, thh, mu1, mu2)
p13 = sum(gamma3(12,i, thh, mu1, mu2) for i in range(1,6))
p23 = gamma3(12,6, thh, mu1, mu2)
p33 = sum(gamma3(12,i, thh, mu1, mu2) for i in range(7,12))
p43 = gamma3(12,12, thh, mu1, mu2)

C = factorial(N)/(factorial(X[0][1])*factorial(X[1][1])*factorial(X[2][1])*factorial(X[3][1])*factorial(X[4][1]))
L_2 = C*p0**X[0][1]*p1**X[1][1]*p0**X[2][1]*p0**X[3][1]*p0**X[4][1]
L_3 = C*p03**X[0][1]*p13**X[1][1]*p23**X[2][1]*p33**X[3][1]*p43**X[4][1]

print(L_2, L_3)

1.46504404761071e-66 7.78665647243216e-53


6. AIC and BIC criteria

In [7]:
AIC_2 = -2*log(L_2) + 2*2
AIC_3 = -2*log(L_3) + 2*3

BIC_2 = -2*log(L_2) + log(225)*2
BIC_3 = -2*log(L_3) + log(225)*3

print(AIC_2, AIC_3)
print(BIC_2.n(), BIC_3.n())

307.177461657923 245.969196737169
314.009662462332 256.217497943782
